In [1]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19 LTS; OpenJDK Runtime Environment Zulu11.56+19-CA (build 11.0.15+10-LTS); OpenJDK 64-Bit Server VM Zulu11.56+19-CA (build 11.0.15+10-LTS, mixed mode)
  Starting server from /opt/homebrew/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/6s/z3758tx16yd17yy35vnsnx_80000gn/T/tmpa8gjd7ym
  JVM stdout: /var/folders/6s/z3758tx16yd17yy35vnsnx_80000gn/T/tmpa8gjd7ym/h2o_davidnunez_started_from_python.out
  JVM stderr: /var/folders/6s/z3758tx16yd17yy35vnsnx_80000gn/T/tmpa8gjd7ym/h2o_davidnunez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,H2O_from_python_davidnunez_05z1hg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
import pandas as pd
y = "Protestas"
balanced_df = pd.read_csv('../balanced_output_35.csv', index_col=False)
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(balanced_df, test_size=0.2, random_state=42)

## Random undersampling

In [4]:
balanced_df_true = balanced_df[balanced_df[y] == True]
balanced_df_false = balanced_df[balanced_df[y] == False]
n_cmp = len(balanced_df_true)
n_no_cmp = len(balanced_df_false)
n_cmp / (n_cmp + n_no_cmp)


0.28513025197872566

In [5]:
# Random undersampling
if True:
	balanced_df_true = balanced_df[balanced_df[y] == True]
	balanced_df_false = balanced_df[balanced_df[y] == False]
	n_cmp = len(balanced_df_true)
	n_no_cmp = len(balanced_df_false)
	total = n_cmp + n_no_cmp
	sample_false = balanced_df_false.sample(int(total*.65)) #Sample 70% no complaints 30 % complaints
	balanced_df = pd.concat([balanced_df_true, sample_false])

## Training

In [6]:
train = h2o.H2OFrame.from_python(balanced_df)
# Identify predictors and response
x = train.columns
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# Run AutoML for 10 base models
aml = H2OAutoML(max_models=42, seed=23, stopping_metric="RMSE", sort_metric="RMSE")
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leader
# board
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |


19:43:39.586: AutoML: XGBoost is not available; skipping it.
19:43:39.594: _train param, Dropping bad and constant columns: [parties.details.legalEntityTypeDetail payer_3, parties.details.legalEntityTypeDetail payer_23, parties.details.legalEntityTypeDetail payer_4, parties.details.legalEntityTypeDetail payer_24, tender.eligibilityCriteria q3, parties.details.legalEntityTypeDetail payer_1, parties.details.legalEntityTypeDetail payer_21, parties.details.legalEntityTypeDetail payer_2, parties.details.legalEntityTypeDetail payer_22, parties.details.legalEntityTypeDetail payer_7, parties.details.legalEntityTypeDetail payer_8, parties.details.legalEntityTypeDetail payer_5, parties.details.legalEntityTypeDetail payer_25, awards.documents.DocumentTypeDetails_19, parties.details.legalEntityTypeDetail payer_6, parties.details.legalEntityTypeDetail tenderer_18, tender.status_planning, parties.details.legalEntityTypeDetail payer_20, parties.details.legalEntityTypeDetail enquirer_18, awards.docum

model_id,rmse,auc,logloss,aucpr,mean_per_class_error,mse
StackedEnsemble_AllModels_1_AutoML_3_20230807_194339,0.182962,0.983056,0.124617,0.975506,0.0591037,0.0334751
StackedEnsemble_BestOfFamily_1_AutoML_3_20230807_194339,0.184518,0.981923,0.127294,0.97437,0.0608187,0.0340469
GBM_grid_1_AutoML_3_20230807_194339_model_11,0.185381,0.981837,0.131171,0.974305,0.0606513,0.0343662
GBM_grid_1_AutoML_3_20230807_194339_model_8,0.185477,0.981817,0.131461,0.97417,0.0598469,0.0344017
GBM_grid_1_AutoML_3_20230807_194339_model_5,0.186096,0.981293,0.13075,0.973568,0.0608341,0.0346316
GBM_grid_1_AutoML_3_20230807_194339_model_7,0.186708,0.981516,0.131089,0.973767,0.0599891,0.0348599
GBM_grid_1_AutoML_3_20230807_194339_model_9,0.187835,0.98117,0.134186,0.973252,0.061636,0.0352822
GBM_1_AutoML_3_20230807_194339,0.188727,0.980981,0.133335,0.972675,0.0614703,0.0356178
GBM_4_AutoML_3_20230807_194339,0.189541,0.980461,0.134166,0.972259,0.0632633,0.035926
GBM_grid_1_AutoML_3_20230807_194339_model_4,0.189666,0.980176,0.136344,0.971799,0.0613508,0.0359734


In [10]:
model_path = h2o.save_model(model=aml.leader, path="train2022/smoteModelv1", force=True)

In [11]:
print(model_path)

/Users/davidnunez/Desktop/tesis/tesis-model/Training/train2022/smoteModelv1/StackedEnsemble_AllModels_1_AutoML_3_20230807_194339


In [12]:
saved_model = h2o.load_model(model_path)

## Testing 2022 Data

In [13]:
df_2023 = pd.read_csv('../df_dummizado_from_2023.csv', index_col=False)
df_2023 = df_2023[df_2023['date.year'] == 2023]
test_2023 = h2o.H2OFrame.from_python(df_2023)
test_2023[y] = test_2023[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
def get_performance(model, test: h2o.H2OFrame):
	cm = model.model_performance(test).confusion_matrix().to_list()
	perf_t = cm[1][1] / (cm[1][0] + cm[1][1])
	perf_f = cm[0][0] / (cm[0][1] + cm[0][0])
	return perf_t, perf_f

In [16]:
best = ''
best_score_t = 0
best_score_f = 0
df_leaderboard = aml.leaderboard.as_data_frame()
model_ids = df_leaderboard['model_id']
for model_id in model_ids:
	model = h2o.get_model(model_id)
	score_t, score_f = get_performance(model, test_2023)
	print(model_id, score_t, score_f)
	if score_t > best_score_t and score_f > best_score_f:
		best_score_t = score_t
		best_score_f = score_f
		best = model_id

print(best, best_score_t, best_score_f)

StackedEnsemble_AllModels_1_AutoML_3_20230807_194339 0.7583019513865115 0.9074834588181611
StackedEnsemble_BestOfFamily_1_AutoML_3_20230807_194339 0.7271482369051695 0.918320784850559
GBM_grid_1_AutoML_3_20230807_194339_model_11 0.7216706607326259 0.9203741729409081
GBM_grid_1_AutoML_3_20230807_194339_model_8 0.7226977062649778 0.9195756331279945
GBM_grid_1_AutoML_3_20230807_194339_model_5 0.7425539198904485 0.9058863791923341
GBM_grid_1_AutoML_3_20230807_194339_model_7 0.7315987675453611 0.9046315309148985
GBM_grid_1_AutoML_3_20230807_194339_model_9 0.7271482369051695 0.9097650011407712
GBM_1_AutoML_3_20230807_194339 0.7110578568983225 0.9273328770248688
GBM_4_AutoML_3_20230807_194339 0.7630948305374872 0.8994980606890258
GBM_grid_1_AutoML_3_20230807_194339_model_4 0.7038685381718589 0.9186630161989505
GBM_grid_1_AutoML_3_20230807_194339_model_6 0.7206436152002739 0.917864476386037
GBM_3_AutoML_3_20230807_194339 0.7237247517973296 0.9237964864248231
GBM_grid_1_AutoML_3_20230807_194339

In [18]:
best_model = h2o.get_model('StackedEnsemble_AllModels_1_AutoML_3_20230807_194339')
best_model.model_performance(test_2023).confusion_matrix()

,False,True,Error,Rate
False,7955.0,811.0,0.0925,(811.0/8766.0)
True,706.0,2215.0,0.2417,(706.0/2921.0)
Total,8661.0,3026.0,0.1298,(1517.0/11687.0)


In [53]:
best_model.model_performance(test_2022).confusion_matrix().to_list()

[[15527, 3139], [1102, 2216]]

In [58]:
model_path = h2o.save_model(model=best_model, path="train2021/smoteModelv8", force=True)

In [55]:
saved_model = h2o.load_model("/Users/matiaslopez/Documents/tesis/tesis-model/Training/train2021/smoteModelv8/DeepLearning_grid_1_AutoML_2_20230803_00151_model_3")

In [56]:
saved_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_2_20230803_00151_model_3


Status of Neuron Layers: predicting Protestas, 2-class classification, bernoulli distribution, CrossEntropy loss, 38,952 weights/biases, 603.6 KB, 4,600,125 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms                momentum    mean_weight            weight_rms          mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ----------------------  ----------  ---------------------  ------------------  --------------------  ------------------
    1        776      Input             10.0
    2        50       RectifierDropout  50.0       0.0   0.0   0.20461730867483416    0.34151124954223633     0.0         0.18682926992339283    0.7766852378845215  0.5849995765635401    0.4228494167327881
    3        2        Softmax                      0.0   0.0   0.0001095066020025115  0.00010655305231921375  0.0         -0.024883554093539716  0.68442702293396    -0.46365044840961833  1.7344741821289062

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.6747781655244295
RMSE: 0.8214488210013022
LogLoss: 4.551511407317617
Mean Per-Class Error: 0.1238022392577428
AUC: 0.8944822404172239
AUCPR: 0.7028411925709815
Gini: 0.7889644808344478

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999525343037174
       False    True    Error    Rate
-----  -------  ------  -------  ---------------
False  6079     984     0.1393   (984.0/7063.0)
True   311      2561    0.1083   (311.0/2872.0)
Total  6390     3545    0.1303   (1295.0/9935.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999953     0.798192  1
max f2                       0.999953     0.851793  1
max f0point5                 0.999953     0.750938  1
max accuracy                 0.999953     0.869653  1
max precision                0.999997     0.72585   0
max recall                   0.795635     1         324
max specificity              0.999997     0.871018  0
max absolute_mcc             0.999953     0.711991  1
max min_per_class_accuracy   0.999953     0.860682  1
max mean_per_class_accuracy  0.999953     0.876198  1
max tns                      0.999997     6152      0
max fns                      0.999997     460       0
max fps                      1.34672e-05  7063      399
max tps                      0.795635     2872      324
max tnr                      0.999997     0.871018  0
max fnr                      0.999997     0.160167  0
max fpr                      1.34672e-05  1         399
max tpr                      0.795635     1         324

Gains/Lift Table: Avg response rate: 28.91 %, avg score: 97.67 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0663312                   1                  3.34903    3.34903            0.968134         1         0.968134                    1                   0.222145        0.222145                   234.903   234.903            0.219172
2        0.100151                    1                  3.3666     3.35496            0.973214         1         0.969849                    1                   0.113858        0.336003                   236.66    235.496            0.331755
3        0.150075                